In [ ]:
import random
import math
import itertools
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
random.seed(0)

In [ ]:
df_training = pd.read_csv('../../datasets/titanic_training_processed.csv')
df_test = pd.read_csv('../../datasets/titanic_test_processed.csv')

This is just an experiment to practice with the concept of statistical distances. The idea is to visualise a dimensionality reduced representation of the two classes (survived / non-survived) to get some awarenenss of their separability and then use a simple Mahalanobis distance between each test observation and the two classes to perform classification. 

Originally I wanted to apply t-SNE for dimensionality reduction, but the t-SNE algorithm in scikit-learn cannot be applied to new data. Therefore, I am sticking with PCA instead. 

In [ ]:
X = df_training.values[:, 2:]
pca = PCA(n_components = 2)
X_pca = pca.fit_transform(X)

In [ ]:
fig, ax = plt.subplots()
_ = ax.scatter(X_pca[df_training.Survived == 1, 0], 
               X_pca[df_training.Survived == 1, 1], 
               c = 'blue', 
               alpha = 0.5)
_ = ax.scatter(X_pca[df_training.Survived == 0, 0], 
               X_pca[df_training.Survived == 0, 1], 
               c = 'red', 
               alpha = 0.5)

There seems to be a large overlap between the rwo distributions. What is the Bhattacharyya distance value?

In [ ]:
def number_observations(array, x, y):
    return np.sum(np.logical_and(array[:, 0] < x, 
                                 np.logical_and(array[:, 0] >= x - 0.1, 
                                                np.logical_and(array[:, 1] < y, 
                                                               array[:, 1] >= y - 0.1))))

In [ ]:
discrete = np.arange(-4, 10, 0.1)
index_1 = df_training.Survived == 1
index_0 = df_training.Survived == 0
n_1 = np.sum(index_1)
n_0 = np.sum(index_0)
bc = 0
for i in list(itertools.product(discrete, discrete)):
    x = i[0]
    y = i[1]
    p_1 = number_observations(X_pca[index_1, :], x, y) / n_1
    p_0 = number_observations(X_pca[index_0, :], x, y) / n_0
    bc = bc + math.sqrt(p_1 * p_0)
print('Bhattacharyya distance: ' + str(-math.log(bc)))